In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("prediction_datasets/one_room/one_room_imp_4_quarter_prediction.csv",parse_dates=[-1], index_col=[0])
df.set_index("Quarter",inplace=True)
df.head()

,00100,00120,00140,00150,00170,00180,00250,00260,00270,00400,...,26100,33200,33710,33720,40100,48100,50100,90100,90120,90500
Quarter,,,,,,,,,,,,,,,,,,,,,
2010-01-01,5458.0,5515.0,5525.0,5549.0,5462.0,5155.0,5084.0,4795.0,5008.0,3769.0,...,1403.0,3200.0,2119.0,2195.0,2398.0,1562.0,1679.0,2500.4,2135.4,1772.4
2010-04-01,5347.0,5646.0,5937.0,5475.0,5852.0,5714.6,4885.0,5433.4,4896.0,3797.0,...,1509.0,3396.0,1997.0,2204.0,2340.0,1396.0,1999.0,2500.4,2252.0,1687.0
2010-07-01,5826.0,5395.0,6082.0,5225.0,5743.0,5677.0,5046.0,5369.0,5004.0,3618.0,...,1638.0,3425.0,1909.0,2186.0,2550.0,1629.0,1835.0,2241.0,2135.4,1777.0
2010-10-01,5566.0,5613.0,5759.0,5348.0,5952.0,5694.8,4867.0,5433.4,5052.0,3716.0,...,1606.0,3281.0,1948.0,2215.0,2320.0,1660.0,2368.0,2500.4,2135.4,1733.0
2011-01-01,5545.0,5905.0,6277.0,5587.0,5886.0,6108.0,5108.0,5408.0,5267.0,3919.0,...,1774.0,3502.0,2249.0,2182.0,2528.0,1712.0,2150.0,2553.0,2161.0,1685.0


In [15]:
# Basic imports
import numpy as np
import pandas as pd
import datetime # manipulating date formats
import itertools

from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings("ignore")

def sarimax_forecasts(df, max_p=2, d=1, max_q=2, max_P=2, D=1, max_Q=2, s=4):
    """
    Keyword arguments:
    multivar_ts -- pandas dataframe of shape (n,m) where n is the number of
    time steps and n is the number of variables for prediction.
    """
    
    res_df = pd.DataFrame(columns=df.columns, index=['pred_0','pred_1','pred_2','pred_3'])
    # set parameter range
    p,d,q = range(0,max_p+1),[d],range(0,max_q+1)
    P,D,Q,s = range(0,max_P+1),[D],range(0,max_Q+1),[s]

    # list of all parameter combos
    pdq = list(map(list,itertools.product(p, d, q)))
    seasonal_pdq = list(map(list, itertools.product(P, D, Q, s)))
    all_param = list(map(list, itertools.product(pdq,seasonal_pdq)))

    count = 0
    for column in df:
        best_res, best_aic = None, np.inf
        for param in all_param:
            try:
                mod = SARIMAX(
                    df[column].values,
                    order=param[0],
                    seasonal_order=param[1]
                )
                
                res = mod.fit(disp=0)
                if res.aic < best_aic:
                    best_res, best_aic = res, res.aic
            except Exception as e:
                print(e)
                continue
        
        count += 1
        # nice = best_res.forecast(steps=4)
        res_df[column] = best_res.forecast(steps=4)

        print(f'The prediction for {column} is:', res_df[column].values)

        print(f"Processed {round(count/len(df.columns)*100,2)}%: {count}/{len(df.columns)}")
            
    return res_df

In [16]:
sarimax_forecasts(df.iloc[:,[0]])

The prediction for 00100 is: [8686.12097272 8582.49246    8841.90816584 9063.901058  ]
Processed 100.0%: 1/1


,00100
pred_0,8686.120973
pred_1,8582.492460
pred_2,8841.908166
pred_3,9063.901058


In [19]:
df = pd.read_csv("prediction_datasets/one_room/one_room_sarimax_forecasts.csv", index_col=[0])
df.head()

,00100,00120
pred_0,8686.120973,8864.809120
pred_1,8582.492460,8352.647861
pred_2,8841.908166,8739.900611
pred_3,9063.901058,8938.433763
